In [1]:
import pandas
from pandas import DataFrame
import sqlite3

<h5>pip install pyArango</h5>

In [2]:
from pyArango.connection import*
from pyArango.query import*
import pyArango.validation as VAL

In [3]:
def show_document(doc):
    print(doc['StockID'])
    print('\tDate=%s\tVolume=%12.2f'%(doc['Date'],doc['Volume']))
    print('\tOpen=%12.2f\tClose=%12.2f\tAdj. Close=%12.2f'%(doc['Open'],doc['Close'],doc['Adj Close']))
    print('\tHigh=%12.2f\tLow=%12.2f'%(doc['High'],doc['Low']))

<h5>ArangoDB 資料庫</h5>

In [4]:
db_conn=Connection(
    arangoURL='http://localhost:8529',
    username='emprogria',
    password='emprogria'
)

In [5]:
db=db_conn['Emprogria'] #轉換名稱
db_collection=db.collections['StockInfo']

<h5>讀取 ArangoDB 資料</h5>

In [8]:
aql='FOR doc IN StockInfo FILTER doc.`Open`> @stockOpen RETURN doc' #@變數(在下方設定變數)

In [9]:
sub_collections=db.AQLQuery(aql, rawResults=False, batchSize=4, bindVars={'stockOpen': 130.9}) #false結構化,資料子集合
print('筆數: %d' % len(sub_collections.response['result']))

筆數: 4


In [10]:
queryResult= sub_collections.response['result']

In [11]:
for doc in queryResult:
    show_document(doc)

IBM
	Date=2017-01-03	Volume=        2.93
	Open=      167.00	Close=      167.19	Adj. Close=      134.70
	High=      167.87	Low=      166.01
IBM
	Date=2017-01-04	Volume=        3.38
	Open=      167.77	Close=      169.26	Adj. Close=      136.37
	High=      169.87	Low=      167.36
IBM
	Date=2017-01-05	Volume=        2.68
	Open=      169.25	Close=      168.70	Adj. Close=      135.92
	High=      169.39	Low=      167.26
IBM
	Date=2017-01-06	Volume=        2.95
	Open=      168.69	Close=      169.53	Adj. Close=      136.58
	High=      169.92	Low=      167.52


In [6]:
aql='FOR doc IN StockInfo FILTER doc.`Open`> @stockOpen AND LEFT(doc.Date,7)==@YM RETURN doc'
sub_collections=db.AQLQuery(aql, rawResults=False, batchSize=4, bindVars={'stockOpen': 130.9, 'YM':'2020-01'})
queryResult= sub_collections.response['result']

In [7]:
for doc in queryResult:
    show_document(doc)

In [12]:
def queryStock(conditions={'stockOpen':130.9,'YM':'2020-01'},numOfRows=4):
    global db
    aql='FOR doc IN StockInfo FILTER doc.`Open`> @stockOpen AND LEFT(doc.Date,7)==@YM RETURN doc'
    sub_collections=db.AQLQuery(aql, rawResults=False, batchSize=numOfRows, bindVars=conditions)
    queryResult= sub_collections.response['result']
    for doc in queryResult:
        show_document(doc)

In [13]:
queryStock(conditions={'stockOpen':135.6,'YM':'2020-02'},numOfRows=8)

In [14]:
def queryStock(conditions={'YM':'2020-01'},numOfRows=31):
    global db
    aql='FOR doc IN StockInfo FILTER LEFT(doc.Date,7)==@YM RETURN doc'
    sub_collections=db.AQLQuery(aql, rawResults=False, batchSize=numOfRows, bindVars=conditions)
    queryResult= sub_collections.response['result']
    
    _resultSet=[]
    
    for doc in queryResult:
        doc['Scale']=doc['High']-doc['Low']
        doc['Net']=doc['Adj Close']-doc['Open']
        _resultSet.append(doc)
    return _resultSet

In [15]:
my_results=DataFrame(data=queryStock(conditions={'YM':'2020-04'},numOfRows=31)) #最多31筆
my_columns=my_results.columns.tolist()

In [16]:
print('\t'.join(my_columns))

In [17]:
my_results[my_columns[3:]].head() #從第三行開始出現

""


In [18]:
print('Scale 平均:%12.2f\t標準差:%12.2f'%(my_results['Scale'].mean(),my_results['Scale'].std()))

KeyError: 'Scale'

In [ ]:
def queryStock(conditions={'YM':'2020-01'},numOfRows=31):
    global db
    aql='FOR doc IN StockInfo FILTER LEFT(doc.Date,7)==@YM RETURN doc'
    sub_collections=db.AQLQuery(aql, rawResults=False, batchSize=numOfRows, bindVars=conditions)
    queryResult= sub_collections.response['result']
    
    _resultSet=[]
    
    for doc in queryResult:
        doc['Scale']=doc['High']-doc['Low']
        doc['Net']=doc['Adj Close']-doc['Open']
        _resultSet.append(doc)
    return DataFrame(data=_resultSet)